In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

customers = pd.read_csv(r'C:/Users/kames/OneDrive/Documents/GitHub/Stout/casestudy.csv')

yearWiseDf = pd.DataFrame()

#1. Total revenue for the current year
totalRevenuePerYear=customers.groupby(['year'], as_index=False).agg(total_revenue = ('net_revenue', 'sum'), customer_count = ('year','count'))
yearWiseDf = totalRevenuePerYear
revenueFig = px.pie(yearWiseDf, values='total_revenue', names='year', title='Total revenue per year')
revenueFig.show()
revenueFig.write_html('docs/case-study-2.1.html')

yearWiseDf['New Customer Revenue'] = yearWiseDf['total_revenue']
yearWiseDf['Existing Customer Growth'] = yearWiseDf['total_revenue']
yearWiseDf['Revenue lost from attrition'] = 0
yearWiseDf['Existing Customer Revenue Current Year'] = yearWiseDf['total_revenue']
yearWiseDf['Existing Customer Revenue Prior Year'] = 0
yearWiseDf['Total Customers Current Year'] = yearWiseDf['customer_count']
yearWiseDf['Total Customers Previous Year'] = 0
yearWiseDf['New Customers'] = yearWiseDf['customer_count']
yearWiseDf['Lost Customers'] = 0

yearWiseDict = dict(tuple(customers.groupby('year')))

df2015 = yearWiseDict[2015]
df2016 = yearWiseDict[2016]
df2017 = yearWiseDict[2017]

df2015 = df2015.drop(df2015.columns[[0, 3]], axis=1)
df2016 = df2016.drop(df2016.columns[[0, 3]], axis=1)
df2017 = df2017.drop(df2017.columns[[0, 3]], axis=1)

#Existing Customer Statistics
common2016_2015 = df2016.merge(df2015, on=["customer_email"])
common2016_2015.rename(columns={'net_revenue_x':'net_revenue_2016', 'net_revenue_y':'net_revenue_2015'}, inplace=True)
common2016_2015['net_growth'] = common2016_2015['net_revenue_2016'] - common2016_2015['net_revenue_2015']

common2016_2015 = common2016_2015.round(2)

existingCustomerNetRevenueCurrentYear2016 = sum(common2016_2015['net_revenue_2016'])
existingCustomerNetRevenuePreviousYear2016 = sum(common2016_2015['net_revenue_2015'])
exisitingCustomerGrowth2016 = existingCustomerNetRevenueCurrentYear2016 - existingCustomerNetRevenuePreviousYear2016;
yearWiseDf.at[1, 'Existing Customer Revenue Current Year'] = existingCustomerNetRevenueCurrentYear2016;
yearWiseDf.at[1, 'Existing Customer Revenue Prior Year'] = existingCustomerNetRevenuePreviousYear2016;
yearWiseDf.at[1,'Existing Customer Growth'] = exisitingCustomerGrowth2016

existingCustomerFig2016 = go.Figure(data=[go.Table(
    header=dict(values=list(common2016_2015.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[common2016_2015.customer_email, common2016_2015.net_revenue_2016, common2016_2015.net_revenue_2015, common2016_2015.net_growth],
               fill_color='lavender',
               align='left'))
])

existingCustomerFig2016.show()
existingCustomerFig2016.write_html('docs/case-study-2.2.html')

common2017_2016 = df2017.merge(df2016, on=["customer_email"])
common2017_2016.rename(columns={'net_revenue_x':'net_revenue_2017', 'net_revenue_y':'net_revenue_2016'}, inplace=True)
common2017_2016['net_growth'] = common2017_2016['net_revenue_2017'] - common2017_2016['net_revenue_2016']
common2017_2016 = common2017_2016.round(2)

existingCustomerNetRevenueCurrentYear2017 = sum(common2017_2016['net_revenue_2017'])
existingCustomerNetRevenuePreviousYear2017 = sum(common2017_2016['net_revenue_2016'])
exisitingCustomerGrowth2017 = existingCustomerNetRevenueCurrentYear2017 - existingCustomerNetRevenuePreviousYear2017;
yearWiseDf.at[2, 'Existing Customer Revenue Current Year'] = existingCustomerNetRevenueCurrentYear2017;
yearWiseDf.at[2, 'Existing Customer Revenue Prior Year'] = existingCustomerNetRevenuePreviousYear2017;
yearWiseDf.at[2,'Existing Customer Growth'] = exisitingCustomerGrowth2017

existingCustomerFig2017 = go.Figure(data=[go.Table(
    header=dict(values=list(common2017_2016.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[common2017_2016.customer_email, common2017_2016.net_revenue_2017, common2017_2016.net_revenue_2016, common2017_2016.net_growth],
               fill_color='lavender',
               align='left'))
])

existingCustomerFig2017.show()
existingCustomerFig2017.write_html('docs/case-study-2.3.html')

yearWiseDf = yearWiseDf.round(2)

existingReveneueFig = go.Figure(data=[
    go.Bar(name='2016', x=['Existing Customer Revenue Current Year', 'Existing Customer Revenue Prior Year', 'Existing Customer Growth'], y=[yearWiseDf.at[1, 'Existing Customer Revenue Current Year'], yearWiseDf.at[1, 'Existing Customer Revenue Prior Year'], yearWiseDf.at[1, 'Existing Customer Growth']]),
    go.Bar(name='2017', x=['Existing Customer Revenue Current Year', 'Existing Customer Revenue Prior Year', 'Existing Customer Growth'], y=[yearWiseDf.at[2, 'Existing Customer Revenue Current Year'], yearWiseDf.at[2, 'Existing Customer Revenue Prior Year'], yearWiseDf.at[2, 'Existing Customer Growth']])
])

existingReveneueFig.update_layout(barmode='group')
existingReveneueFig.show()
existingReveneueFig.write_html('docs/case-study-2.4.html')


#New Customer Statistics

diff2016_2015 = df2016[~df2016.customer_email.isin(common2016_2015.customer_email)]
diff2017_2016 = df2017[~df2017.customer_email.isin(common2017_2016.customer_email)]

diff2016_2015 = diff2016_2015.round(2)
diff2017_2016 = diff2017_2016.round(2)

newCustomerRevenue2016 = sum(diff2016_2015.net_revenue)
newCustomerRevenue2017 = sum(diff2017_2016.net_revenue)

newCustomerFig2016 = go.Figure(data=[go.Table(
    header=dict(values=list(diff2016_2015.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[diff2016_2015.customer_email, diff2016_2015.net_revenue],
               fill_color='lavender',
               align='left'))
])

newCustomerFig2016.show()
newCustomerFig2016.write_html('docs/case-study-2.5.html')

newCustomerFig2017 = go.Figure(data=[go.Table(
    header=dict(values=list(diff2017_2016.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[diff2017_2016.customer_email, diff2017_2016.net_revenue],
               fill_color='lavender',
               align='left'))
])

newCustomerFig2017.show()
newCustomerFig2017.write_html('docs/case-study-2.6.html')

yearWiseDf.at[1, 'New Customers'] = diff2016_2015.shape[0];
yearWiseDf.at[2, 'New Customers'] = diff2017_2016.shape[0];
yearWiseDf.at[1,'New Customer Revenue'] = newCustomerRevenue2016
yearWiseDf.at[2,'New Customer Revenue'] = newCustomerRevenue2017

yearWiseDf = yearWiseDf.round(2)

newCustomerFig = px.scatter(yearWiseDf, x="year", y="New Customer Revenue",
	         size="New Customers", size_max=60)
newCustomerFig.update_xaxes(type='category')
newCustomerFig.show()
newCustomerFig.write_html('docs/case-study-2.7.html')

yearWiseDf.at[1, 'Total Customers Previous Year'] = yearWiseDf.at[0, 'customer_count']
yearWiseDf.at[2, 'Total Customers Previous Year'] = yearWiseDf.at[1, 'customer_count']

#Lost customers statistics

diff2015_2016 = df2015[~df2015.customer_email.isin(common2016_2015.customer_email)]
diff2016_2017 = df2016[~df2016.customer_email.isin(common2017_2016.customer_email)]

diff2015_2016 = diff2015_2016.round(2)
diff2016_2017 = diff2016_2017.round(2)

revenueLostFromAttrition2016 = sum(diff2015_2016.net_revenue)
revenueLostFromAttrition2017 = sum(diff2016_2017.net_revenue)

yearWiseDf.at[1, 'Lost Customers'] = diff2015_2016.shape[0]
yearWiseDf.at[2, 'Lost Customers'] = diff2016_2017.shape[0]
yearWiseDf.at[1, 'Revenue lost from attrition'] = revenueLostFromAttrition2016
yearWiseDf.at[2, 'Revenue lost from attrition'] = revenueLostFromAttrition2017

yearWiseDf = yearWiseDf.round(2)

lostCustomerFig = px.scatter(yearWiseDf, x="year", y="Revenue lost from attrition",
	         size="Lost Customers", size_max=60)
lostCustomerFig.update_xaxes(type='category')
lostCustomerFig.show()
lostCustomerFig.write_html('docs/case-study-2.8.html')

customerFig = go.Figure(data=[
    go.Bar(name='2015', x=['Total Customers Current Year', 'Total Customers Previous Year', 'New Customers', 'Lost Customers'], y=[yearWiseDf.at[0, 'Total Customers Current Year'], yearWiseDf.at[0, 'Total Customers Previous Year'], yearWiseDf.at[0, 'New Customers'], yearWiseDf.at[0, 'Lost Customers']]),
    go.Bar(name='2016', x=['Total Customers Current Year', 'Total Customers Previous Year', 'New Customers', 'Lost Customers'], y=[yearWiseDf.at[1, 'Total Customers Current Year'], yearWiseDf.at[1, 'Total Customers Previous Year'], yearWiseDf.at[1, 'New Customers'], yearWiseDf.at[1, 'Lost Customers']]),
    go.Bar(name='2017', x=['Total Customers Current Year', 'Total Customers Previous Year', 'New Customers', 'Lost Customers'], y=[yearWiseDf.at[2, 'Total Customers Current Year'], yearWiseDf.at[2, 'Total Customers Previous Year'], yearWiseDf.at[2, 'New Customers'], yearWiseDf.at[2, 'Lost Customers']])
])

customerFig.update_layout(barmode='group')
customerFig.show()
customerFig.write_html('docs/case-study-2.9.html')

yearWiseFig = go.Figure(data=[go.Table(
    header=dict(values=list(yearWiseDf.columns),
                line_color='darkslategray',
                fill_color='lightskyblue',
                align='left'),
    cells=dict(values=[yearWiseDf['year'], yearWiseDf['total_revenue'], yearWiseDf['customer_count'], yearWiseDf['New Customer Revenue'],
      yearWiseDf['Existing Customer Growth'], yearWiseDf['Revenue lost from attrition'],
       yearWiseDf['Existing Customer Revenue Current Year'],
       yearWiseDf['Existing Customer Revenue Prior Year'], yearWiseDf['Total Customers Current Year'],
       yearWiseDf['Total Customers Previous Year'], yearWiseDf['New Customers'], yearWiseDf['Lost Customers']],
               line_color='darkslategray',
               fill_color='lightcyan',
               align='left'))
])

yearWiseFig.show()
yearWiseFig.write_html('docs/case-study-2.html')



In [ ]:

finalFig = px.line(yearWiseDf, x='year', y="total_revenue")
finalFig.show()
finalFig.write_html('docs/case-study-2a.html')

In [ ]:
finalFig = px.line(yearWiseDf, x='year', y="Existing Customer Revenue Current Year")
finalFig.show()
finalFig.write_html('docs/case-study-2b.html')

In [ ]:
finalFig = px.line(yearWiseDf, x='year', y="Existing Customer Revenue Prior Year")
finalFig.show()
finalFig.write_html('docs/case-study-2c.html')

In [ ]:
finalFig = px.line(yearWiseDf, x='year', y="Existing Customer Growth")
finalFig.show()
finalFig.write_html('docs/case-study-2d.html')

In [ ]:
finalFig = px.line(yearWiseDf, x='year', y="New Customers")
finalFig.show()
finalFig.write_html('docs/case-study-2e.html')

In [13]:
finalFig = px.line(yearWiseDf, x='year', y="Lost Customers")
finalFig.show()
finalFig.write_html('docs/case-study-2f.html')